# Load Training Data

In [1]:
# These libraries are pre-installed on SageMaker notebook instances.
import boto3, json, io, pandas as pd

In [2]:
s3_uri = "s3://echodata25/results/echo-images/nova-pro/gen9-outputs/job_00/te63ogaqhzx0/prompts.jsonl.out"
bucket, key = s3_uri.replace("s3://", "", 1).split("/", 1)

In [3]:
import boto3, math
s3 = boto3.client("s3")

s3_uri = "s3://echodata25/results/echo-images/nova-pro/gen9-outputs/job_00/te63ogaqhzx0/prompts.jsonl.out"
bucket, key = s3_uri.replace("s3://", "", 1).split("/", 1)

size = s3.head_object(Bucket=bucket, Key=key)["ContentLength"]   # bytes
print(f"{size/2**30:,.2f} GiB  ({size:,} bytes)")


0.66 GiB  (703,444,983 bytes)
